##### Here I'm using findspark to indicate where the spark module is located

In [1]:
import findspark
findspark.init(r'C:\Users\q1011812\Downloads\spark-3.0.1-bin-hadoop3.2\spark-3.0.1-bin-hadoop3.2')

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('LG-project').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression
dataset = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)
dataset.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [4]:
dataset.show(4)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
+----------------+----+----

In [5]:
dataset.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [6]:
dataset = dataset.select(['Age','Total_Purchase','Account_Manager','Years','Num_Sites','Churn'])
dataset.show(5)

+----+--------------+---------------+-----+---------+-----+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|
+----+--------------+---------------+-----+---------+-----+
|42.0|       11066.8|              0| 7.22|      8.0|    1|
|41.0|      11916.22|              0|  6.5|     11.0|    1|
|38.0|      12884.75|              0| 6.67|     12.0|    1|
|42.0|       8010.76|              0| 6.71|     10.0|    1|
|37.0|       9191.58|              0| 5.56|      9.0|    1|
+----+--------------+---------------+-----+---------+-----+
only showing top 5 rows



In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
assembler = VectorAssembler(inputCols = ['Age','Total_Purchase','Years','Num_Sites','Account_Manager'], outputCol = 'features') 

In [9]:
output = assembler.transform(dataset)
final_data = output.select('features','churn')
final_data.head(1)

[Row(features=DenseVector([42.0, 11066.8, 7.22, 8.0, 0.0]), churn=1)]

In [10]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [11]:
training_data, test_data = final_data.randomSplit([0.7,0.3])

In [12]:
from pyspark.ml.classification import LogisticRegression
lr_churn = LogisticRegression(labelCol='churn')
fitted_churn_model = lr_churn.fit(training_data)
training_sum = fitted_churn_model.summary
training_sum.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              churn|         prediction|
+-------+-------------------+-------------------+
|  count|                646|                646|
|   mean|0.17337461300309598|0.13003095975232198|
| stddev| 0.3788641599512765| 0.3365981188504344|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [14]:
pred_and_labels = fitted_churn_model.evaluate(test_data)
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[26.0,8939.61,4.5...|    0|[6.38273611913967...|[0.99831236170014...|       0.0|
|[28.0,8670.98,3.9...|    0|[7.76753627656195...|[0.99957692430281...|       0.0|
|[28.0,11128.95,5....|    0|[4.06028830645042...|[0.98304826960409...|       0.0|
|[29.0,13255.05,4....|    0|[4.07622680730614...|[0.98331183984323...|       0.0|
|[30.0,8677.28,7.3...|    0|[3.98573803158191...|[0.98176014570988...|       0.0|
|[30.0,10183.98,5....|    0|[2.73085032328089...|[0.93882269342212...|       0.0|
|[30.0,10960.52,5....|    0|[2.26381387348027...|[0.90583544916739...|       0.0|
|[31.0,5387.75,6.8...|    0|[2.45732301824875...|[0.92109532238612...|       0.0|
|[31.0,10182.6,3.7...|    0|[4.62596622008900...|[0.99030080857360...|       0.0|
|[31.0,11297.57,

In [15]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='churn')
auc = churn_eval.evaluate(pred_and_labels.predictions)

In [17]:
final_lr_model = lr_churn.fit(final_data)
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,
                              header=True)
new_customers.printSchema()
test_new_customers = assembler.transform(new_customers)
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [18]:
final_results = final_lr_model.transform(test_new_customers)
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

